## 5章章末問題

In [1]:
import pandas as pd
import statsmodels.api as sm
import os
os.makedirs('./output', exist_ok=True)

---
(1) 4.9節で実装した V/P戦略を思い出そう。そのときは，CAPMにもとづいて各銘柄の期待リターンを推定した。ここでは，CAPMに代わり，FF3で期待リターンを推定し，理論株価を評価してみよう。月間のマーケットリスクプレミアムは 0.3%，サイズ・プレミアムは0.1%，バリュー・プレミアムは0.5%とする。そして，V/Pが1を上回る割安株銘柄のみを抽出し，向こう1年間について，等加重，ならびに，時価総額加重で運用した場合のそれぞれについて，FF3アルファを推定してみよう。

---

In [2]:
# 以下、コード4.1〜4.4を再掲し、それらのコードを元に改修している。
# 途中経過を表示するprint()はコメントアウトしている。

####
# コード 4.1 ライブラリの読み込みと入力データの準備

# 各銘柄のβ推定などに利用するffMonthly.csvを読み込み
ffMonthly = pd.read_csv('./data/ffMonthly.csv', parse_dates=['month'])
ffMonthly['month'] = ffMonthly['month'].dt.to_period('M')
# print(ffMonthly)
##        month   RMRF   SMB   HML    RF
## 0    1990-07  20.67 -1.56 -5.16  0.68
## 1    1990-08 -13.69 -3.63  0.98  0.66

# 必要なデータの選択(行と列の選択)
# 産業A(tickerがAから始まる) の月次リターンデータを読み込み
stockMonthly = pd.read_csv('./data/stockMonthly.csv', parse_dates=['month'])
stockMonthly['month'] = stockMonthly['month'].dt.to_period('M')
stockMonthly = stockMonthly[stockMonthly['industry'] == 'A']

# β推定のために，2008/7-2013/6の60ヶ月のデータを選択
df = stockMonthly[(stockMonthly['month'] >= '2008-7') &
                  (stockMonthly['month'] <= '2013-6')].copy()
df = df[['ticker', 'month', 'close', 'return', 'share']]
# ffMonthlyを結合し各銘柄の月次超過リターン (RIRF)を求める
df = pd.merge(df, ffMonthly[['month', 'RMRF', 'RF', 'SMB', 'HML']], on='month')  # => SMB, HMLもmergeしておく
df['RIRF'] = df['return'] - df['RF']
# print(df)
#      ticker    month  close     return      share   RMRF    RF   SMB   HML       RIRF  
# 0     A0001  2008-07   2820  -7.571288   15815814   3.24  0.15  0.65 -0.56  -7.721288  
# 1     A0002  2008-07  12484  16.270840    5046567   3.24  0.15  0.65 -0.56  16.120840  

# V/P推定後の運用期間データの選択
futureRet = stockMonthly[(stockMonthly['month'] >= '2013-7') &
                         (stockMonthly['month'] <= '2014-6')]
futureRet = futureRet[['ticker', 'month', 'return']]
# print(futureRet)
##       ticker    month     return
## 270    A0001  2013-07  22.149300
## 271    A0001  2013-08   8.260577

# 各銘柄の1期先の予想配当データの読み込み
fy1D = pd.read_csv('./data/dividendData.csv')
# print(fy1D)
##    ticker  fy1Dividend
## 0   A0001         52.0
## 1   A0002        148.0

######
# コード 4.2 ticker別にベータを推定する
# 単回帰でβを推定する関数  => CAPMでなくFF3に変更
def calBeta(d):
    model = sm.OLS(d['RIRF'], sm.add_constant(d[['RMRF', 'SMB', 'HML']]))
    res = model.fit()  # OLSによる回帰係数の推定
    return res.params[['RMRF', 'SMB', 'HML']]  # 推定された定数項とRMRFの回帰係数

# print(df[['ticker', 'RIRF', 'RMRF']])
##      ticker       RIRF   RMRF
## 0     A0001  -7.721288   3.24
## 1     A0002  16.120840   3.24

# 銘柄ごとにβを推定
beta = df[['ticker', 'RIRF', 'RMRF', 'SMB', 'HML']].groupby('ticker').apply(calBeta)
beta.columns = ['RMRF_coef', 'SMB_coef', 'HML_coef']
print(beta)
#         RMRF_coef  SMB_coef  HML_coef
# ticker                               
# A0001    1.010250 -0.196170  0.622242
# A0002    0.533288  1.485306 -0.368115


####
# コード 4.3 2013年6月の株価データから割安株を選択するプログラム

# ポートフォリを組成する銘柄の選択と投資ウェイトの計算
# 各銘柄の2013年6月末時点のV/Pを計算するため，2013-06を選択，予想配当やβも結合
df2 = df[df['month'] == '2013-06'].copy()
df2 = pd.merge(df2, fy1D, on='ticker')
df2 = pd.merge(df2, beta, on='ticker')

# 各銘柄のV/Pの推定値 (hatVP)の計算
# 年間の期待リターン（企業にとっての株式の資本コスト）単位は(%)
# マーケットリスクプレミアムは月間 0.3%とする  # => ここをFF3の全ファクタを対応させる
df2['expRet'] = (df2['RF'] + df2['RMRF_coef'] * 0.3 + df2['SMB_coef'] * 0.1+ df2['HML_coef'] * 0.5) * 12
# 毎期fy1Dividendの配当が永続すると仮定した理論株価
df2['hatV'] = df2['fy1Dividend'] / (df2['expRet'] / 100)
df2['hatVP'] = df2['hatV'] / df2['close']  # 2013年6月末時点で推定されたV/P

# hatVPが1を上回る割安株だけを抽出
df2 = df2[df2['hatVP'] > 1.0]
print('len(df2):', len(df2))  ## 13

# 抽出された割安株で構成されたポートフォリオを等加重と時価総額加重で運用するための投資ウェイトを計算しておく
df2['ew'] = 1.0 / len(df2)  # 等加重で運用する場合の各銘柄に対する投資ウェイト
df2['me'] = df2['close'] * df2['share']  # 各銘柄の時価総額 (= 株価 × 発行済み株式数)
df2['vw'] = df2['me'] / df2['me'].sum()  # 時価総額加重で運用する場合の各銘柄に対する投資ウェイト
# print(df2)

####
# コード4.4 ポートフォリオの評価

# 割安株で構成されたポートフォリオの運用結果
# 運用期間のデータに等加重と時価総額加重の場合のそれぞれの投資ウェイトを結合し，等加重平均リターンと加重平均リターンをそれぞれ求める
# tickerをキーに結合
futureRet2 = pd.merge(futureRet, df2[['ticker', 'ew', 'vw']], on='ticker')

futureRet2['ewRet'] = futureRet2['return'] * futureRet2['ew']
futureRet2['vwRet'] = futureRet2['return'] * futureRet2['vw']
print(futureRet2)
#     ticker    month     return        ew        vw     ewRet     vwRet
# 0    A0009  2013-07  -2.175970  0.076923  0.000921 -0.167382 -0.002003
# 1    A0009  2013-08  14.410060  0.076923  0.000921  1.108466  0.013265

# 上で計算されたewRetとvwRetを月ごとに足し合わせれば完成
rsl = futureRet2[['month', 'ewRet', 'vwRet']].groupby('month').sum()
print(rsl)
##              ewRet      vwRet
## month
## 2013-07  -2.828633   4.815620
## 2013-08   6.110663  11.472223

# FF3アルファを推定するため，rslとffMonthlyを結合し，等加重に基づく超過リターンをewRPRF，時価総額加重に基づく超過リターンをvwRPRFとして求める
df3 = pd.merge(rsl, ffMonthly[['month', 'RMRF', 'RF', 'SMB', 'HML']], on='month')  
df3['ewRPRF'] = df3['ewRet'] - df3['RF']
df3['vwRPRF'] = df3['vwRet'] - df3['RF']
print(df3)
##       month     ewRet     vwRet  RMRF   RF   SMB   HML    ewRPRF    vwRPRF
## 0   2013-07 -0.544932 -4.975972 -0.36  0.0 -2.07 -1.09 -0.544932 -4.975972
## 1   2013-08  6.652734  3.947967  5.07  0.0 -0.97  1.12  6.652734  3.947967

# FF3アルファの推定
# 等加重の場合
ewModel = sm.OLS(df3['ewRPRF'], sm.add_constant(df3[['RMRF', 'SMB', 'HML']]))
ewRes = ewModel.fit()  # OLSによる推定
print(ewRes.summary())  # 推定結果を表示

# 時価総額加重の場合
vwModel = sm.OLS(df3['vwRPRF'], sm.add_constant(df3[['RMRF', 'SMB', 'HML']]))
vwRes = vwModel.fit()  # OLSによる推定
print(vwRes.summary())  # 推定結果を表示

        RMRF_coef  SMB_coef  HML_coef
ticker                               
A0001    1.010250 -0.196170  0.622242
A0002    0.533288  1.485306 -0.368115
A0003    1.682153  1.289306  0.531089
A0004    1.172929  1.387567  2.355616
A0005    0.854129 -0.050156  1.662248
...           ...       ...       ...
A0076    0.748343  0.853158  0.691442
A0077    0.307402  0.283402  1.454854
A0078    1.010900  0.842200  1.358183
A0079    0.549189  0.561207  0.052458
A0080    0.805557  0.409757  1.454905

[77 rows x 3 columns]
len(df2): 13
    ticker    month     return        ew        vw     ewRet     vwRet
0    A0009  2013-07  -2.175970  0.076923  0.000921 -0.167382 -0.002003
1    A0009  2013-08  14.410060  0.076923  0.000921  1.108466  0.013265
2    A0009  2013-09   4.057481  0.076923  0.000921  0.312114  0.003735
3    A0009  2013-10   4.305443  0.076923  0.000921  0.331188  0.003963
4    A0009  2013-11  -2.180685  0.076923  0.000921 -0.167745 -0.002007
..     ...      ...        ...       ...    

/Users/muramiya/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/muramiya/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


---
(2) stockMonthly.csvに収録されているqmeの列は，各年6月末の時価総額にもとづく5分位(ME1はサイズがもっとも小さい銘柄群で，ME5がサイズがもっとも大きい銘柄群)，qbemeは各年6月末のBE/MEにもとづく5分位(BM1はBE/MEがもっとも低い銘柄群で，BM5がBE/MEがもっとも高い銘柄群)を表している。計25個のSize-BE/MEポートフォリオについて，各年6月末の時価総額(close×share)にもとづいて，月ごとに時価加重平均リターンを求め，次のようなdfを作成してみよう。なお，時価加重平均リターンを求める期間は，2000年7月から2014年6月までの計168ヶ月とする。

|年月 (month)|サイズの5分位 (qme)|BE/MEの5分位 (qbeme)|月次tの加重平均リターン($R_{p,t}$)|
|:------:|:---:|:-----:|:--------:|
|2000-07 | ME1 | BM1   | 3.566904 |
|2000-07 | ME1 | BM2   | 0.6535981|
|2000-07 | ME1 | BM3   | 0.8231361|
|2000-07 | ME1 | BM4   | 1.058473 |
|2000-07 | ME1 | BM5   | 3.681175 |
|2000-07 | ME2 | BM1   | 1.20337  |
|  ...   | ... | ...   | ...      |
|2014-06 | ME5 | BM4   | 7.623873 |
|2014-06 | ME5 | BM5   | 11.43039 |

---

In [3]:
# stockMonthly.csvの読み込み
stockMonthly = pd.read_csv('./data/stockMonthly.csv', parse_dates=['month'])
stockMonthly['month'] = stockMonthly['month'].dt.to_period('M')
# print(stockMonthly)
#        ticker    month  open  high   low  close   volume     share     return  industry  qme qbeme
# 0       A0001  1991-01  1571  1605  1503   1533   109323  19856748   4.285714         A  ME1   BM5 
# 1       A0001  1991-02  1503  1516  1457   1503   108565  19856748  -1.956947         A  ME1   BM5 

# 各年6月末の時価総額を求める
juneME = stockMonthly[stockMonthly['month'].dt.month==6].copy()
juneME['me'] = juneME['close'] * juneME['share']
juneME = juneME[['ticker', 'month', 'me']]
# 6月の時価総額を翌月(7月)から適用するので月だけを+1して7月にしておく。
juneME['month'] = juneME['month'] + 1
# print(juneME)
#        ticker    month           me
# 5       A0001  1991-07  26905893540
# 17      A0001  1992-07  20710588164

df = stockMonthly[['ticker', 'month', 'return', 'qme', 'qbeme']].copy()
# left-outer結合で、7月以外のme列はNAとなる。
df = df.merge(juneME, on=['ticker', 'month'], how='left')
# me列のNAを直前の値で埋める(ffill)ことで7月に結合されたMEが翌月以降次の6月まで反映されることになる。
df['me'] = df.groupby('ticker')['me'].fillna(method='ffill')
# print(df.head(20))
#    ticker    month     return  qme qbeme            me
#       :        :          :     :     :             :
# 16  A0001  1992-05  -3.703704  ME1   BM5  2.690589e+10
# 17  A0001  1992-06   2.859961  ME1   BM5  2.690589e+10
# 18  A0001  1992-07   7.382550  ME2   BM5  2.071059e+10
# 19  A0001  1992-08   8.303572  ME2   BM5  2.071059e+10

# 月,qme,qbeme別に時価総額ウェイトリターンを求める
def func(d):
    # print(d)
    # ticker    month     return  qme qbeme           me
    # 34     A0003  1995-06  -7.241379  ME1   BM1  18937475184
    # 195    A0011  1991-06  -2.512255  ME1   BM1   8091442569    
    
    # 時価総額加重で運用する場合の各銘柄に対する投資ウェイト
    d['vw'] = d['me'] / d['me'].sum()
    
    # 時価総額加重で運用した場合のリターン
    d['vwRet'] = d['vw'] * d['return']
    return d[['vwRet']].sum()

res = df.groupby(['month', 'qme', 'qbeme']).apply(func)
print(res)
#                        vwRet
# month   qme qbeme           
# 1991-01 ME1 BM1     0.000000
#             BM2     0.000000

res = res.reset_index()
res = res[(res['month']>='2000-07') & (res['month']<='2014-06')]
print(res)
#         month  qme qbeme     vwRet
# 2850  2000-07  ME1   BM1  2.118126
# 2851  2000-07  ME1   BM2  0.466493
# 2852  2000-07  ME1   BM3  0.472636
# 2853  2000-07  ME1   BM4  0.817654
# 2854  2000-07  ME1   BM5  4.120662
# ...       ...  ...   ...       ...
# 7045  2014-06  ME5   BM1  7.824797
# 7046  2014-06  ME5   BM2  7.724612
# 7047  2014-06  ME5   BM3  8.322632
# 7048  2014-06  ME5   BM4  6.686710
# 7049  2014-06  ME5   BM5  9.325307


                       vwRet
month   qme qbeme           
1991-01 ME1 BM1     0.000000
            BM2     0.000000
            BM3     0.000000
            BM4     0.000000
            BM5     0.000000
...                      ...
2014-12 ME5 BM1     8.797871
            BM2     8.826196
            BM3     9.653725
            BM4    11.488360
            BM5    10.717793

[7200 rows x 1 columns]
        month  qme qbeme     vwRet
2850  2000-07  ME1   BM1  2.118126
2851  2000-07  ME1   BM2  0.466493
2852  2000-07  ME1   BM3  0.472636
2853  2000-07  ME1   BM4  0.817654
2854  2000-07  ME1   BM5  4.120662
...       ...  ...   ...       ...
7045  2014-06  ME5   BM1  7.824797
7046  2014-06  ME5   BM2  7.724612
7047  2014-06  ME5   BM3  8.322632
7048  2014-06  ME5   BM4  6.686710
7049  2014-06  ME5   BM5  9.325307

[4200 rows x 4 columns]


---
(3) 計25個のSize-BE/ME ポートフォリオのそれぞれについて，CAPMを前提として(5.7)式を推定し，$\hat{\alpha}_{p}^{\text{CAPM}}$と$\hat{\beta}_{p}$，並びにそれぞれの$t$値を5$\times$5の行列にまとめてエクスポートしてみよう．

---

In [4]:
df = res.merge(ffMonthly[['month', 'RF', 'RMRF', 'SMB', 'HML']], on='month')
df['RPRF'] = df['vwRet'] - df['RF']
print(df)

def ols(d):
    # 各ポートフォリオの回帰モデル
    model = sm.OLS(d['RPRF'], sm.add_constant(d[['RMRF']]))
    res = model.fit()
    coef = res.params # 係数
    coef.index = ['alpha', 'beta']
    tval = res.tvalues  # t値
    tval.index = ['alpha_t', 'beta_t']
    return pd.concat([coef, tval])


# ポートフォリオごとに回帰モデルを推定し，alpha,betaの係数とt値を推定
res2 = df.groupby(['qme', 'qbeme']).apply(ols)
print(res2)

# pivotを使って5x5のデータ形式にしてエクスポート
alpha = res2.reset_index().pivot(index='qme', columns='qbeme', values='alpha')
print('\nCAPM alpha')
print(alpha)
alpha.to_csv('./output/5-Q_CAPMalpha.csv')

beta = res2.reset_index().pivot(index='qme', columns='qbeme', values='beta')
print('\nCAPM beta')
print(beta)
beta.to_csv('./output/5-Q_CAPMbeta.csv')

alpha_t = res2.reset_index().pivot(index='qme', columns='qbeme', values='alpha_t')
print('\nalpha t値')
print(alpha_t)
alpha_t.to_csv('./output/5-Q_CAPMalpha_t.csv')

beta_t = res2.reset_index().pivot(index='qme', columns='qbeme', values='beta_t')
print('\nbeta t値')
print(beta_t)
beta_t.to_csv('./output/5-Q_CAPMbeta_t.csv')


        month  qme qbeme     vwRet    RF  RMRF   SMB   HML      RPRF
0     2000-07  ME1   BM1  2.118126  0.48 -1.54  3.22  1.17  1.638126
1     2000-07  ME1   BM2  0.466493  0.48 -1.54  3.22  1.17 -0.013507
2     2000-07  ME1   BM3  0.472636  0.48 -1.54  3.22  1.17 -0.007364
3     2000-07  ME1   BM4  0.817654  0.48 -1.54  3.22  1.17  0.337654
4     2000-07  ME1   BM5  4.120662  0.48 -1.54  3.22  1.17  3.640662
...       ...  ...   ...       ...   ...   ...   ...   ...       ...
4195  2014-06  ME5   BM1  7.824797  0.00  8.02  3.33  1.04  7.824797
4196  2014-06  ME5   BM2  7.724612  0.00  8.02  3.33  1.04  7.724612
4197  2014-06  ME5   BM3  8.322632  0.00  8.02  3.33  1.04  8.322632
4198  2014-06  ME5   BM4  6.686710  0.00  8.02  3.33  1.04  6.686710
4199  2014-06  ME5   BM5  9.325307  0.00  8.02  3.33  1.04  9.325307

[4200 rows x 9 columns]
              alpha      beta   alpha_t     beta_t
qme qbeme                                         
ME1 BM1    0.541173  0.840379  1.461786  11.3

---
(4) 今度は，FF3を前提に，(5.8)式を推定し，$\hat{\alpha}_{p}^{\text{FF3}}$，$\hat{b}_{p}$，$\hat{s}_{p}$，$\hat{h}_{p}$，並びにそれぞれの$t$統計量を5$\times$5の行列にまとめてエクスポートし，CAPMを前提とした場合とFama-French 3ファクター・モデルを前提とした場合で，アルファの出方に違いが出るか否か調べてみよう．

---

In [5]:
def ols(d):
    # FF3アルファを推定する回帰モデル
    model = sm.OLS(d['RPRF'], sm.add_constant(d[['RMRF', 'SMB', 'HML']]))
    res = model.fit()
    coef = res.params # 係数
    coef.index = ['alpha', 'b', 's', 'h']
    tval = res.tvalues  # t値
    tval.index = ['alpha_t', 'b_t', 's_t', 'h_t']
    return pd.concat([coef, tval])


# ポートフォリオごとに回帰モデルを推定し，alpha,betaの係数とt値を推定
res3 = df.groupby(['qme', 'qbeme']).apply(ols)
print(res3)

# pivotを使って5x5のデータ形式にしてエクスポート（b, s, hのエクスポートについては同様の手順のため，以下ではFF3アルファの手順のみを示す）
alpha = res3.reset_index().pivot(index='qme', columns='qbeme', values='alpha')
print('\nFF3 alpha')
print(alpha)
alpha.to_csv('./output/5-Q_FF3alpha.csv')

alpha_t = res3.reset_index().pivot(index='qme', columns='qbeme', values='alpha_t')
print('\nalpha t値')
print(alpha_t)
alpha_t.to_csv('./output/5-Q_FF3alpha_t.csv')

              alpha         b         s         h   alpha_t        b_t  \
qme qbeme                                                                
ME1 BM1    0.166494  1.114616  1.346003  0.207885  0.749112  24.935372   
    BM2    0.056126  0.929037  1.067927  0.175591  0.374769  30.844464   
    BM3    0.239784  0.883143  1.003338  0.268624  1.735579  31.783179   
    BM4    0.159453  0.862472  0.968283  0.286484  1.350665  36.324830   
    BM5    0.156186  0.855005  0.989107  0.442811  1.688909  45.969990   
ME2 BM1    0.006031  1.108770  1.213207  0.031511  0.035689  32.621242   
    BM2   -0.106441  0.969644  0.962819  0.219089 -0.736447  33.357030   
    BM3   -0.009849  0.889105  0.800730  0.325233 -0.096036  43.104876   
    BM4    0.062641  0.957233  0.872913  0.566419  0.517004  39.282257   
    BM5   -0.029821  0.952224  0.899917  0.705936 -0.405715  64.413836   
ME3 BM1    0.001753  1.115678  0.987341 -0.062914  0.009837  31.123304   
    BM2   -0.057459  0.979033  0.69310